# 1 - Two-agent Chat

## Setup

Material de referencia: https://autogen-ai.github.io/autogen/docs/topics/llm_configuration/#adding-http-client-in-llm_config-for-proxy

In [ ]:
import os

config_list = [
    {
        "model": "gemma2:2b",
        "base_url": "http://localhost:11434/v1",
        "api_key": "ollama",
    },{
        "model": "llama-3.1-70b-versatile",
        "base_url": "https://api.groq.com/openai/v1",  
        "api_key": os.getenv("GROQ_API_KEY"),  
    }
]

llm_config = {"config_list":config_list}

## Definindo um agente

In [ ]:
from autogen import ConversableAgent

agent = ConversableAgent(
    name="agente_1",
    llm_config=llm_config["config_list"][1],  # Acessando o primeiro item da lista dentro do dicionário
    human_input_mode="NEVER",
)

In [ ]:
reply = agent.generate_reply(
    messages=[{"content": "Qual o seu modelo de LLM? Responda em PT-BR", "role": "user"}]
)
print(reply)

Definindo os agentes para conversação:

In [ ]:
pesquisador = ConversableAgent(
    name="pesquisador",
    system_message=
    "Voce é um agente de IA pesquisador de conteudo."
    "Seu trabalho é pesquisar as principais noticias da semana "
    "na area de IA."
    "Suas respostas devem ser em português PT-BR.",
    llm_config=llm_config["config_list"][0],
    human_input_mode="NEVER",
)

In [ ]:
reply = pesquisador.generate_reply(
    messages=[{"content": "Qual a sua role? No que voce pode me ajudar?", "role": "user"}]
)
print(reply)

In [ ]:
sumarizador = ConversableAgent(
    name="sumarizador",
    system_message=
    "Voce é um agente de IA especialista em criar resumos."
    "Voce ira trabalhar junto do agente pesquisador para "
    "obter os dados que ele coletou e sumarizar."
    "Suas respostas devem ser em português PT-BR.",
    llm_config=llm_config["config_list"][1],
    human_input_mode="NEVER",
)

In [ ]:
reply = sumarizador.generate_reply(
    messages=[{"content": "Qual a sua role? No que voce pode me ajudar?", "role": "user"}]
)
print(reply)

Iniciando a conversação

In [ ]:
chat_result = sumarizador.initiate_chat(
    recipient=pesquisador, 
    message="Por favor, pesquise na internet as principais noticias da semana sobre Inteligencia Artificial",
    max_turns=2,
)

## Visualizando alguns resultados

Voce pode visualizar:

1. Historico do Chat
2. Custo
3. Summario da conversa

In [ ]:
import pprint

pprint.pprint(chat_result.chat_history)

In [ ]:
pprint.pprint(chat_result.cost)

In [ ]:
pprint.pprint(chat_result.summary)

## Tenha uma melhor sumarização

In [ ]:
chat_result = sumarizador.initiate_chat(
    pesquisador, 
    message="Olá! Faça uma pesquisa na internet com  "
    "as ultimas noticias de semana sobre inteligencia artificial, Gen AI, agentic workflows", 
    max_turns=3, 
    summary_method="reflection_with_llm",
    summary_prompt="Resuma a conversa e entregue o resultado em PT-BR e bullet points",
)

In [ ]:
pprint.pprint(chat_result.summary)

## Encerrando o chat

O chat pode ser encerrado passando uma condição de finalização

In [ ]:
assistente = ConversableAgent(
    name="assistente",
    system_message=
    "Voce é um assistente pessoal baseado em IA Generativa e irá auxiliar "
    "a criar uma planejamento de estudos de Agentic Workflow", 
    llm_config=llm_config["config_list"][1],
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "Isso é tudo, obrigado" in msg["content"],
)

planejador = ConversableAgent(
    name="planejador",
    system_message=
    "Voce é um planejador especialista em Ultra Learnig. "
    "Crie um plano de estudos para aprendizado de Agentic Workflow"
    "Quando voce estiver pronto para encerrar a conversa, diga 'Isso é tudo, obrigado'",
    llm_config=llm_config["config_list"][0],
    human_input_mode="NEVER",
)

In [ ]:
chat_result = planejador.initiate_chat(
    recipient=assistente,
    message="Olá, eu sou o planejador, me traga os assuntos mais importantes para estudo de IA Generativa e agentic workflow."
)

In [ ]:
pprint.pprint(chat_result.cost)